In [88]:
import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from git import Repo
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language
from langchain.vectorstores import Chroma
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [89]:
load_dotenv()


True

In [90]:
## load the GROQ And OpenAI API KEY 
groq_api_key=os.getenv('GROQ_API_KEY')
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="gemma-7b-it")

In [91]:
prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)

In [92]:

%pwd

'd:\\GENAI\\SourceCodeAnalysisAgent\\research'

In [93]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [94]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/VinitPahwa1985/StockMarketApi", to_path=repo_path)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/VinitPahwa1985/StockMarketApi test_repo/
  stderr: 'fatal: destination path 'test_repo' already exists and is not an empty directory.
'

In [95]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/src',
                                        glob = "**/*",
                                       suffixes=[".java"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [96]:
documents = loader.load()

In [97]:
documents

[Document(page_content='package com.vinit.stockmarket.stockmarket;\n\nimport org.springframework.boot.SpringApplication;\nimport org.springframework.boot.autoconfigure.SpringBootApplication;\n\n@SpringBootApplication\npublic class StockmarketApplication {\n\n\tpublic static void main(String[] args) {\n\t\tSpringApplication.run(StockmarketApplication.class, args);\n\t}\n\n}\n', metadata={'source': 'test_repo\\src\\main\\java\\com\\vinit\\stockmarket\\stockmarket\\StockmarketApplication.java', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='package com.vinit.stockmarket.stockmarket.controller;\n\nimport java.net.URI;\nimport java.sql.Timestamp;\nimport java.util.ArrayList;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.Map;\nimport java.util.Optional;\n\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.data.domain.Page;\nimport org.springframework.data.domain.PageRequest;\nimport org.springframework.data.d

In [ ]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.JAVA,
                                                             chunk_size = 1000,
                                                             chunk_overlap = 200)

In [ ]:
texts = documents_splitter.split_documents(documents)

In [ ]:
len(texts)

0

In [ ]:
embeddings=GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [ ]:
vectordb = FAISS.from_documents(texts, embedding=embeddings)


IndexError: list index out of range

In [ ]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

In [ ]:
question = "what is StockResourceController ?"

In [ ]:
result = qa(question)
print(result['answer'])

The provided text does not include the conversation or the follow up question, so I am unable to extract the necessary information to complete the request.


In [ ]:
import time



if question:
    document_chain=create_stuff_documents_chain(llm,prompt)
    retriever=vectordb.as_retriever()
    retrieval_chain=create_retrieval_chain(retriever,document_chain)
    start=time.process_time()
    response=retrieval_chain.invoke({'input':question})
    print("Response time :",time.process_time()-start)
    print(response['answer'])


Response time : 0.171875
**StockResourceController** is a controller class in the given context. It provides an API endpoint to retrieve stock information from a database. 

- The controller is annotated with `@RestController`, indicating it returns JSON responses.
- It uses the `StockJpaRepository` bean to interact with a database repository for accessing `Stock` entities.
- The `getAllStocks` method retrieves all stock information from the database and returns them as a JSON response.
